In [1]:
#Import relevant libraries
%matplotlib qt
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sb
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


from scipy import stats
from scipy.special import boxcox, inv_boxcox
from scipy.spatial.distance import euclidean

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import optimizers
from tensorflow.keras.models import clone_model

from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV

# Database tables

### Episódios

In [2]:
df = pd.read_excel('data/DW_Pneumonia_episodios.xls')
print("Number of Columns: ",df.shape[1])
print("Number of Rows: ",df.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  102
Number of Rows:  15355


In [3]:
df[df['VentilacaoInvasiva'] == 1]

,Key,SourceKey,Idioma,HospitalRegiaoId,HospitalCodigo,ServicoId,DoenteId,DoenteDataNascimento,DoenteIdadeEmAnos,DoenteDecadas,...,NrPrescricoesBroncodilatador,Psicofarmaco,NrPrescricoesPsicofarmaco,Opiaceo,NrPrescricoesOpiaceo,RelaxanteMuscular,NrPrescricoesRelaxanteMuscular,Antibiotico,NrPrescricoesAntibiotico,LineageKey
0,1,3005,PT,3,15,3,2383,1951-09-30,66,7,...,6,0,0,1,2,0,0,1,8,1
1,2,3006,PT,3,15,3,2384,1946-10-23,71,8,...,0,1,1,1,2,1,1,1,5,1
2,3,3012,PT,3,15,3,2390,1964-01-22,54,6,...,4,1,2,1,1,0,0,1,3,1
3,4,3013,PT,3,15,1,2391,1956-05-31,61,7,...,0,1,1,1,2,1,3,1,4,1
4,5,3019,PT,3,15,2,2397,1944-07-20,73,8,...,1,0,0,0,0,0,0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15332,15333,8311,PT,3,11,103,6910,1955-04-07,64,7,...,0,1,2,0,0,0,0,1,3,36
15333,15334,8353,PT,3,11,103,6944,1997-09-18,22,3,...,0,1,1,0,0,0,0,1,2,36
15334,15335,8483,PT,3,11,103,7057,1940-06-26,79,8,...,1,1,1,0,0,1,1,1,4,36
15347,15348,9109,PT,3,11,103,7569,1939-04-14,80,9,...,0,1,12,0,0,0,0,1,2,36


In [4]:
def getCount(col):
    d_count = {}
    for i, num in enumerate(df[col]):
        h_code = df["HospitalCodigo"][i]
        s = str(num)+"."+str(h_code)
        s = float(s)
        if s not in list(d_count.keys()):
            d_count[s] = 1
        else:
            d_count[s] += 1
    return dict(sorted(d_count.items(), key=lambda item: item[1], reverse=True))

In [5]:
prov = getCount('DoenteId')
print(len(prov))
print(prov)


13591
{2833.9: 6, 5762.21: 5, 10633.21: 5, 5113.3: 5, 7523.3: 5, 2617.35: 5, 494.9: 5, 1314.9: 5, 1687.9: 5, 4237.13: 5, 4335.13: 5, 403.11: 5, 5749.21: 4, 11375.21: 4, 4942.21: 4, 9121.21: 4, 4974.21: 4, 8143.21: 4, 6272.3: 4, 1902.3: 4, 6983.3: 4, 7102.3: 4, 7469.3: 4, 3818.4: 4, 2828.35: 4, 3888.35: 4, 2735.35: 4, 2835.35: 4, 504.9: 4, 633.9: 4, 4124.9: 4, 1067.9: 4, 7237.9: 4, 283.9: 4, 5422.9: 4, 2844.25: 4, 756.25: 4, 676.25: 4, 331.13: 4, 1317.13: 4, 2855.13: 4, 589.7: 4, 3568.8: 4, 11413.8: 4, 8367.8: 4, 8410.8: 4, 2078.11: 4, 1448.15: 3, 4013.15: 3, 812.15: 3, 981.15: 3, 1306.15: 3, 5255.15: 3, 5309.15: 3, 2509.21: 3, 2599.21: 3, 5335.21: 3, 3654.21: 3, 11427.21: 3, 16449.21: 3, 6505.21: 3, 11986.21: 3, 13498.21: 3, 18091.21: 3, 9478.21: 3, 14447.21: 3, 13564.21: 3, 19332.21: 3, 19375.21: 3, 19448.21: 3, 19186.21: 3, 19269.21: 3, 10050.21: 3, 15406.21: 3, 15545.21: 3, 2348.21: 3, 5126.21: 3, 5458.21: 3, 15768.21: 3, 10527.21: 3, 3634.21: 3, 12506.21: 3, 12388.21: 3, 12756.21: 

In [6]:
df_tipo_servico = pd.read_excel('data/DW_Pneumonia_TipoServico.xls')
print("Number of Columns: ",df_tipo_servico.shape[1])
print("Number of Rows: ",df_tipo_servico.shape[0])
df_tipo_servico.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  103


,Key,HospitalCodigo,SourceKey,Nome,NomeBreve,TipoServicoId,LineageKey
0,1,15,1,A,A,1,1
1,2,15,3,B,B,1,1
2,3,15,4,C,C,1,1
3,4,15,5,D,D,3,1
4,5,15,6,E,E,3,1


### Prone Position

In [7]:
df_pos_prone_24h = pd.read_excel('data/DW_Pneumonia_PosicaoProne_24h.xls')
print("Number of Columns: ",df_pos_prone_24h.shape[1])
print("Number of Rows: ",df_pos_prone_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  508


### Glasgow Score

In [8]:
df_glasgow_24h_min = pd.read_excel('data/DW_Pneumonia_Glasgow_24h_min.xls')
print("Number of Columns: ",df_glasgow_24h_min.shape[1])
print("Number of Rows: ",df_glasgow_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  6
Number of Rows:  42170


### Dispositivos

In [9]:
df_dispositivosI_24h = pd.read_excel('data/DW_Pneumonia_Dispositivos_I_24h.xls')
print("Number of Columns: ",df_dispositivosI_24h.shape[1])
print("Number of Rows: ",df_dispositivosI_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  47693


In [10]:
df_dispositivosI_24h = pd.read_excel('data/DW_Pneumonia_Dispositivos_I_24h_new.xls')
print("Number of Columns: ",df_dispositivosI_24h.shape[1])
print("Number of Rows: ",df_dispositivosI_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  33530


In [11]:
df_dispositivosII_24h = pd.read_excel('data/DW_Pneumonia_Dispositivos_II_24h.xls')
print("Number of Columns: ",df_dispositivosII_24h.shape[1])
print("Number of Rows: ",df_dispositivosII_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  24715


In [12]:
df_dispositivosII_24h = pd.read_excel('data/DW_Pneumonia_Dispositivos_II_24h_new.xls')
print("Number of Columns: ",df_dispositivosII_24h.shape[1])
print("Number of Rows: ",df_dispositivosII_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  38249


In [13]:
df_tubo_orotraqueal_24h = pd.read_excel('data/DW_Pneumonia_dispositivos_TuboOrotraqueal.xls')
print("Number of Columns: ",df_tubo_orotraqueal_24h.shape[1])
print("Number of Rows: ",df_tubo_orotraqueal_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  6579


### VMI

In [14]:
df_ventilacao_24h = pd.read_excel('data/DW_Pneumonia_Ventilacao_24h.xls')
print("Number of Columns: ",df_ventilacao_24h.shape[1])
print("Number of Rows: ",df_ventilacao_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  8
Number of Rows:  5847


### Tecnica de substituição Renal

In [15]:
df_subs_renal_24h = pd.read_excel('data/DW_Pneumonia_Tecn_Renal_24h.xls')
print("Number of Columns: ",df_subs_renal_24h.shape[1])
print("Number of Rows: ",df_subs_renal_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  654


### Prescrições

In [16]:
df_prescricoes_24h = pd.read_excel('data/DW_Pneumonia_Prescricoes_24h_wout0.xls')
print("Number of Columns: ",df_prescricoes_24h.shape[1])
print("Number of Rows: ",df_prescricoes_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  12
Number of Rows:  64558


In [17]:
df_saps = pd.read_excel('data/DW_Pneumonia_SAPS.xls')
print("Number of Columns: ",df_saps.shape[1])
print("Number of Rows: ",df_saps.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  28
Number of Rows:  11878


In [18]:
df_diagn = pd.read_excel('data/DW_Pneumonia_Diagnosticos.xls')
print("Number of Columns: ",df_diagn.shape[1])
print("Number of Rows: ",df_diagn.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  62955


In [19]:
df_antecedent = pd.read_excel('data/DW_Pneumonia_Antecedentes_Diagn.xls')
print("Number of Columns: ",df_antecedent.shape[1])
print("Number of Rows: ",df_antecedent.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  14132


In [20]:
df_tabagismo_ant = pd.read_excel('data/DW_Pneumonia_Tabagismo_Diagn_antec.xls')
print("Number of Columns: ",df_tabagismo_ant.shape[1])
print("Number of Rows: ",df_tabagismo_ant.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  6
Number of Rows:  762


In [21]:
df_alcoolismo_ant = pd.read_excel('data/DW_Pneumonia_Alcoolismo_Diagn_antec.xls')
print("Number of Columns: ",df_alcoolismo_ant.shape[1])
print("Number of Rows: ",df_alcoolismo_ant.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  6
Number of Rows:  372


In [22]:
df_traqueostomia_24h = pd.read_excel('data/DW_Pneumonia_Traqueostomia_24h.xls')
print("Number of Columns: ",df_traqueostomia_24h.shape[1])
print("Number of Rows: ",df_traqueostomia_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  315


In [23]:
df_infections_24h = pd.read_excel('data/DW_Pneumonia_Infections_24h.xls')
print("Number of Columns: ",df_infections_24h.shape[1])
print("Number of Rows: ",df_infections_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  12
Number of Rows:  13871


In [24]:
df_infections_pneu = pd.read_excel('data/DW_Pneumonia_infecoes_pneumonia.xls')
print("Number of Columns: ",df_infections_pneu.shape[1])
print("Number of Rows: ",df_infections_pneu.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  12
Number of Rows:  8585


In [25]:
"""
    Only espisodes with patient diagnosed with pneumonia at Admission
"""
def OnlyPneuAdmission(epis_frame, inf_frame):
    admiss_pneu_eps = list(inf_frame['EpisodioKey'])
    admiss_pneu_eps_index = [x-1 for x in admiss_pneu_eps]
    admiss_pneu_eps_index = list(dict.fromkeys(admiss_pneu_eps_index))  # remove duplicates
    epis_frame = epis_frame.iloc[admiss_pneu_eps_index]
    return epis_frame

In [26]:
df_doenca_resp = pd.read_excel('data/DW_Pneumonia_Doenca_respiratoria.xls')
print("Number of Columns: ",df_doenca_resp.shape[1])
print("Number of Rows: ",df_doenca_resp.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  11
Number of Rows:  1787


In [27]:
df_diabetes = pd.read_excel('data/DW_Pneumonia_Diabetes_Diagn.xls')
print("Number of Columns: ",df_diabetes.shape[1])
print("Number of Rows: ",df_diabetes.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  11
Number of Rows:  1207


In [28]:
df_doenca_circ = pd.read_excel('data/DW_Pneumonia_Doenca_circulatoria.xls')
print("Number of Columns: ",df_doenca_circ.shape[1])
print("Number of Rows: ",df_doenca_circ.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  11
Number of Rows:  4884


### Sinais Vitais Enfermagem - 24h

In [29]:
df_sinais_enf_FC_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_FC.xls')
print("Number of Columns: ",df_sinais_enf_FC_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_FC_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  22753


In [30]:
df_sinais_enf_FC_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_FC_MAX.xls')
print("Number of Columns: ",df_sinais_enf_FC_24h_max.shape[1])
print("Number of Rows: ",df_sinais_enf_FC_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  14228


In [31]:
df_sinais_enf_FR_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_FR.xls')
print("Number of Columns: ",df_sinais_enf_FR_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_FR_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  30070


In [32]:
df_sinais_enf_FR_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_FR_MAX.xls')
print("Number of Columns: ",df_sinais_enf_FR_24h_max.shape[1])
print("Number of Rows: ",df_sinais_enf_FR_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13773


In [33]:
df_sinais_enf_TAS_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_TAS.xls')
print("Number of Columns: ",df_sinais_enf_TAS_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_TAS_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  23349


In [34]:
df_sinais_enf_TAD_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_TAD.xls')
print("Number of Columns: ",df_sinais_enf_TAD_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_TAD_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  23340


In [35]:
df_sinais_enf_TAM_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_TAM.xls')
print("Number of Columns: ",df_sinais_enf_TAM_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_TAM_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  22633


In [36]:
df_sinais_enf_TAM_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_TAM_MIN.xls')
print("Number of Columns: ",df_sinais_enf_TAM_24h_min.shape[1])
print("Number of Rows: ",df_sinais_enf_TAM_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13886


In [37]:
df_sinais_enf_SpO2_24h = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_SpO2.xls')
print("Number of Columns: ",df_sinais_enf_SpO2_24h.shape[1])
print("Number of Rows: ",df_sinais_enf_SpO2_24h.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  7
Number of Rows:  22350


In [38]:
df_sinais_enf_SpO2_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_SpO2_MIN.xls')
print("Number of Columns: ",df_sinais_enf_SpO2_24h_min.shape[1])
print("Number of Rows: ",df_sinais_enf_SpO2_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  14204


In [39]:
df_sinais_enf_FiO2_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_FiO2_MIN.xls')
print("Number of Columns: ",df_sinais_enf_FiO2_24h_min.shape[1])
print("Number of Rows: ",df_sinais_enf_FiO2_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  9168


In [40]:
df_sinais_enf_Temp_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_enfermagem_Temp_MAX.xls')
print("Number of Columns: ",df_sinais_enf_Temp_24h_max.shape[1])
print("Number of Rows: ",df_sinais_enf_Temp_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13867


### Sinais Vitais Monitor - 24h

In [41]:
df_sinais_monitor_FC_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_FC_MAX.xls')
print("Number of Columns: ",df_sinais_monitor_FC_24h_max.shape[1])
print("Number of Rows: ",df_sinais_monitor_FC_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13214


In [42]:
df_sinais_monitor_FR_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_FR_MAX.xls')
print("Number of Columns: ",df_sinais_monitor_FR_24h_max.shape[1])
print("Number of Rows: ",df_sinais_monitor_FR_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  12600


In [43]:
df_sinais_monitor_Temp_24h_max = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_Temp_MAX.xls')
print("Number of Columns: ",df_sinais_monitor_Temp_24h_max.shape[1])
print("Number of Rows: ",df_sinais_monitor_Temp_24h_max.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  2405


In [44]:
df_sinais_monitor_TAM_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_TAM_MIN.xls')
print("Number of Columns: ",df_sinais_monitor_TAM_24h_min.shape[1])
print("Number of Rows: ",df_sinais_monitor_TAM_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13161


In [45]:
df_sinais_monitor_SpO2_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_SpO2_MIN.xls')
print("Number of Columns: ",df_sinais_monitor_SpO2_24h_min.shape[1])
print("Number of Rows: ",df_sinais_monitor_SpO2_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  13078


In [46]:
df_sinais_monitor_FiO2_24h_min = pd.read_excel('data/DW_Pneumonia_Sinais_monitor_FiO2_MIN.xls')
print("Number of Columns: ",df_sinais_monitor_FiO2_24h_min.shape[1])
print("Number of Rows: ",df_sinais_monitor_FiO2_24h_min.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  3
Number of Rows:  2974


### Gasimetrias

In [47]:
df_gas_be = pd.read_excel('data/DW_Pneumonia_Gasimetria_BE.xls')
print("Number of Columns: ",df_gas_be.shape[1])
print("Number of Rows: ",df_gas_be.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  7917


In [48]:
df_gas_bicarbonato = pd.read_excel('data/DW_Pneumonia_Gasimetria_Bicarbonato.xls')
print("Number of Columns: ",df_gas_bicarbonato.shape[1])
print("Number of Rows: ",df_gas_bicarbonato.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  13101


In [49]:
df_gas_fio2 = pd.read_excel('data/DW_Pneumonia_Gasimetria_FiO2.xls')
print("Number of Columns: ",df_gas_fio2.shape[1])
print("Number of Rows: ",df_gas_fio2.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  7973


In [50]:
df_gas_lactacto = pd.read_excel('data/DW_Pneumonia_Gasimetria_Lactacto.xls')
print("Number of Columns: ",df_gas_lactacto.shape[1])
print("Number of Rows: ",df_gas_lactacto.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  11301


In [51]:
df_gas_methb = pd.read_excel('data/DW_Pneumonia_Gasimetria_metHb.xls')
print("Number of Columns: ",df_gas_methb.shape[1])
print("Number of Rows: ",df_gas_methb.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  8460


In [52]:
df_gas_paco2 = pd.read_excel('data/DW_Pneumonia_Gasimetria_PaCO2.xls')
print("Number of Columns: ",df_gas_paco2.shape[1])
print("Number of Rows: ",df_gas_paco2.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  13206


In [53]:
df_gas_pao2 = pd.read_excel('data/DW_Pneumonia_Gasimetria_PaO2.xls')
print("Number of Columns: ",df_gas_pao2.shape[1])
print("Number of Rows: ",df_gas_pao2.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  13208


In [54]:
df_gas_ph = pd.read_excel('data/DW_Pneumonia_Gasimetria_pH.xls')
print("Number of Columns: ",df_gas_ph.shape[1])
print("Number of Rows: ",df_gas_ph.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  13206


In [55]:
df_gas_sao2 = pd.read_excel('data/DW_Pneumonia_Gasimetria_SaO2.xls')
print("Number of Columns: ",df_gas_sao2.shape[1])
print("Number of Rows: ",df_gas_sao2.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  5
Number of Rows:  12964


### Rastreios

In [56]:
df_rastreios_micro_positivo = pd.read_excel('data/DW_Pneumonia_RastreioMicroPositivo.xls')
print("Number of Columns: ",df_rastreios_micro_positivo.shape[1])
print("Number of Rows: ",df_rastreios_micro_positivo.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  12
Number of Rows:  142


In [57]:
df_rastreios_sero_positivo = pd.read_excel('data/DW_Pneumonia_RastreioSeroPositivo.xls')
print("Number of Columns: ",df_rastreios_sero_positivo.shape[1])
print("Number of Rows: ",df_rastreios_sero_positivo.shape[0])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Number of Columns:  10
Number of Rows:  625


# Data processing

In [58]:
"""
    Specific Infections
"""
def addInfections(data_frame):
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        infec_name = data_frame['InfecaoDescrPT'][_index]
        _index_eps = eps_list.index(ep)
        if infec_name not in df_v2.columns:
            df_v2[infec_name] = 0
            df_v2.at[_index_eps,infec_name]=1
        else:
            df_v2.at[_index_eps,infec_name]=1

In [59]:
"""
    Specific Devices
"""
def addAllDevices(df1, df2):
    eps_list = list(df['Key'])
    for _index, ep in enumerate(df1['EpisodioKey']):
        disp_name = df1['DispositivoDescrPT'][_index]
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            if disp_name not in df_v2.columns:
                df_v2[disp_name] = 0
                df_v2.at[_index_eps,disp_name]=1
            else:
                df_v2.at[_index_eps,disp_name]=1
    
    for _index, ep in enumerate(df2['EpisodioKey']):
        disp_name = df2['DispositivoDescrPT'][_index]
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            if disp_name not in df_v2.columns:
                df_v2[disp_name] = 0
                df_v2.at[_index_eps,disp_name]=1
            else:
                df_v2.at[_index_eps,disp_name]=1

In [60]:
"""
    Have pneumonia at admission
"""
def addPneumoniaAdmission(data_frame):
    df_v2['PneumoniaAdmission'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            df_v2.at[_index_eps,'PneumoniaAdmission'] = 1

In [61]:
"""
    if the admission had infections
"""
def addInfectionAdmission(data_frame):
    df_v2['InfectionAdmission'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            df_v2.at[_index_eps,'InfectionAdmission'] = 1

In [62]:
"""
    Se o paciente usou um dispositivo
"""
def addDevice(data_frameI, data_frameII, col):
    df_v2[col] = 0
    eps_list1 = data_frameI[data_frameI['DispositivoDescrPT']==col].groupby('EpisodioKey').apply(pd.DataFrame)
    eps_list1 = list(eps_list1['EpisodioKey'])
    eps_list2 = data_frameII[data_frameII['DispositivoDescrPT']==col].groupby('EpisodioKey').apply(pd.DataFrame)
    eps_list2 = list(eps_list2['EpisodioKey'])
    for _index, ep in enumerate(df['Key']):
        if ep in eps_list1:
            df_v2.at[_index,col]=1
        elif ep in eps_list2:
            df_v2.at[_index,col]=1

In [63]:
"""
    Glasgow
"""
def addTipoPrescricoes(data_frame):
    df_v2['Corticosteróide'] = 0
    df_v2['Antimalárico'] = 0
    df_v2['Antivírico'] = 0
    df_v2['Anticoagulante'] = 0
    df_v2['Vasopressor'] = 0
    df_v2['Broncodilatador'] = 0
    df_v2['Psicofármaco'] = 0
    df_v2['Opiáceo'] = 0
    df_v2['Relaxante muscular'] = 0
    df_v2['Antibiótico'] = 0
    
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            tipo_presc = data_frame['TipoPrescricaoDescrPT'][_index]
            #print(ep,_index_eps)
            df_v2.at[_index_eps, tipo_presc] += 1

In [64]:
"""
    NumPresc
"""
def addNumPresc(data_frame):
    df_v2['NumPrescricoes'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            df_v2.at[_index_eps, 'NumPrescricoes'] += 1

In [65]:
"""
    NumPresc
"""
def addDuracaoPresc(data_frame):
    df_v2['DuracaoPrescricoes'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            duracao_presc = data_frame['DuracaoEmDias'][_index]
            df_v2.at[_index_eps, 'DuracaoPrescricoes'] += duracao_presc

In [66]:
"""
    NumPresc
"""
def addFactorPresc(data_frame):
    df_v2['FactorPrescricoes'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        _index_eps = eps_list.index(ep)
        dose_presc = data_frame['Factor'][_index]
        df_v2.at[_index_eps, 'FactorPrescricoes'] += dose_presc

In [67]:
"""
    Glasgow
"""
def addGlasgowScore(data_frame):
    df_v2['GlasgowScoreMin'] = np.nan
    eps_list = list(data_frame['EpisodioKey'])
    for _index, ep in enumerate(df['Key']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            df_v2.at[_index,'GlasgowScoreMin']=data_frame['Score'][_index_eps]

In [68]:
"""
    Ventilação
"""
def addVMI(data_frame):
    df_v2['VMI'] = 0
    eps_list = list(df['Key'])
    for _index, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index_eps = eps_list.index(ep)
            if data_frame['TipoVentilacaoId'][_index] == 1:
                df_v2.at[_index_eps,'VMI']=1

In [69]:
"""
    Substituicao renal
"""
def addSubsRenal(data_frame):
    df_v2['SubsRenal'] = 0
    eps_list = list(data_frame['EpisodioKey'])
    for _index, ep in enumerate(df['Key']):
        if ep in eps_list:
            df_v2.at[_index,'SubsRenal']=1

In [70]:
"""
    Posição Prone
"""
def addPosicaoProne(data_frame):
    df_v2['PosicaoProne'] = 0
    eps_list = list(data_frame['EpisodioKey'])
    for _index, ep in enumerate(df['Key']):
        if ep in eps_list:
            df_v2.at[_index,'PosicaoProne']=1

In [71]:
"""
    Numero de Dispositivos  df_dispositivosI
"""
def addDispositivosNum(data_frameI, data_frameII):
    
    df_v2['DispositivosNum'] = 0
    eps_list1 = list(data_frameI['EpisodioKey'])
    eps_list2 = list(data_frameII['EpisodioKey'])
    for _index, ep in enumerate(df['Key']):
        if ep in eps_list1 or ep in eps_list2:
            df_v2.at[_index,'DispositivosNum']=eps_list1.count(ep) + eps_list2.count(ep)

In [72]:
"""
    Add the number of Chronic desease Ratio to Episodes
"""
def addSAPSmetrics(col):
    feature_saps = 'SAPS'+col
    df_v2[feature_saps] = np.nan
    eps_list = list(df_saps['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        if ep in eps_list:
            _index = eps_list.index(ep)
            df_v2.at[_index_eps,feature_saps]=df_saps[col][_index]

In [73]:
"""
    Add the Episode Type to Episodes
"""
def addTipoServico():
    df_v2['TipoServico'] = np.nan
    serv_list = list(df_tipo_servico['Key'])
    for _index_eps, serv in enumerate(df['ServicoId']):    
        if serv in serv_list:
            _index = serv_list.index(serv)
            df_v2.at[_index_eps,'TipoServico']=df_tipo_servico['TipoServicoId'][_index]

In [74]:
"""
    Add the Episode Type to Episodes
"""
def addTipoInfecao(data_frame):
    df_v2['TipoInfecao'] = np.nan
    ep_list = list(data_frame['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):    
        if ep in ep_list:
            _index = ep_list.index(ep)
            df_v2.at[_index_eps,'TipoInfecao']=data_frame['TipoInfecaoId'][_index]

In [75]:
"""
    Add the Infections to Episodes
"""
def addNumeroDiagnAntec():
    df_v2['NumeroDiagnAntec'] = 0
    eps_list = list(df_antecedent['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        df_v2.at[_index_eps,'NumeroDiagnAntec'] = eps_list.count(ep)

In [76]:
"""
    Add the Infections to Episodes
"""
def addNumeroDiagn():
    df_v2['NumeroDiagn'] = 0
    eps_list = list(df_diagn['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        df_v2.at[_index_eps,'NumeroDiagn'] = eps_list.count(ep)

In [77]:
"""
    Check antecedentes de Doenca do Sistema Respiratorio Diagnosis
"""
def addDiseaseAntec(data_frame, col):
    if col not in list(data_frame.columns):
        df_v2[col] = 0
    for _index_eps, ep in enumerate(df['Key']):
        ep_query = df_antecedent.loc[df_antecedent['EpisodioKey'] == ep]
        diagnisticos = list(ep_query["DiagnosticoId"])
        doenca_ids = list(data_frame["Key"])
        for d in diagnisticos:
            if d in doenca_ids:
                df_v2.at[_index_eps,col] += 1

In [78]:
def addAlcoTaba(data_frame, col):
    if col not in list(df_v2.columns):
        df_v2[col] = 0
    eps_list = list(df['Key'])
    for _index_eps, ep in enumerate(data_frame['EpisodioKey']):
        if ep in eps_list:
            _index = eps_list.index(ep)
            df_v2.at[_index, col] += 1


In [79]:
"""
    Check antecedentes de Doenca do Sistema Respiratorio Diagnosis
"""
def addDiabetesAntec():
    df_v2['DiabetesAntec'] = 0
    for _index_eps, ep in enumerate(df['Key']):
        ep_query = df_antecedent.loc[df_antecedent['EpisodioKey'] == ep]
        diagnisticos = list(ep_query["DiagnosticoId"])
        diabetes_ids = list(df_diabetes["Key"])
        for d in diagnisticos:
            if d in diabetes_ids:
                df_v2.at[_index_eps,'DiabetesAntec']=1

In [80]:
"""
    Check antecedentes de Doenca do Sistema Respiratorio Diagnosis
"""
def addDoencaCirculatoriaAntec():
    df_v2['DoencaCirculatoriaAntec'] = 0
    for _index_eps, ep in enumerate(df['Key']):
        ep_query = df_antecedent.loc[df_antecedent['EpisodioKey'] == ep]
        diagnisticos = list(ep_query["DiagnosticoId"])
        doenca_circ_ids = list(df_doenca_circ["Key"])
        for d in diagnisticos:
            if d in doenca_circ_ids:
                df_v2.at[_index_eps,'DoencaCirculatoriaAntec'] += 1

In [81]:
"""
    Add the Infections to Episodes
"""
def addInfectionsNumber(data_frame):
    df_v2['InfectionsNumber'] = 0
    inf_list = list(data_frame['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        df_v2.at[_index_eps,'InfectionsNumber'] = inf_list.count(ep)

In [82]:
def addSepsis(data_frame):
    df_v2['NoSepsis'] = 0
    df_v2['Sepsis'] = 0
    df_v2['SevereSepsis'] = 0
    df_v2['ShockSepsis'] = 0
    for _index_eps, ep in enumerate(df['Key']):
        ep_query = data_frame.loc[data_frame['EpisodioKey'] == ep]
        sepsis_list = list(ep_query["SepsisId"])
        if 1 in sepsis_list:
                df_v2.at[_index_eps,'NoSepsis']=1
        if 2 in sepsis_list:
                df_v2.at[_index_eps,'Sepsis']=1
        if 3 in sepsis_list:
                df_v2.at[_index_eps,'SevereSepsis']=1
        if 4 in sepsis_list:
                df_v2.at[_index_eps,'ShockSepsis']=1
                #df_v2.at[_index_eps,'SepsisGrave']=df_v2.at[_index_eps,'SepsisGrave'] + s

In [83]:
def addBacteriemia(data_frame):
    df_v2['Bacteriemia'] = 0
    for _index_eps, ep in enumerate(df['Key']):
        ep_query = data_frame.loc[data_frame['EpisodioKey'] == ep]
        bact_list = list(ep_query["BacteriemiaSecundariaId"])
        if 1 in bact_list:
            df_v2.at[_index_eps,'Bacteriemia'] = 1

In [84]:
def addVitalSign(df_mon, df_enf, col):
    df_v2[col] = np.nan
    mon_eps_list = list(df_mon['EpisodioKey'])
    enf_eps_list = list(df_enf['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        if ep in mon_eps_list:
            _index = mon_eps_list.index(ep)
            df_v2.at[_index_eps,col]=df_mon['Valor'][_index]

        elif ep in enf_eps_list:
            _index = enf_eps_list.index(ep)
            df_v2.at[_index_eps,col]=df_enf['Valor'][_index]

In [85]:
def addElectiveAdmission(data_frame):
    df_v2['ElectiveAdmission'] = 0
    for _index_eps, ep in enumerate(data_frame['Key']):
        admis_type = data_frame['TipoAdmissaoId'][_index_eps]
        if admis_type == 4 or admis_type == 14 or admis_type == 17:
            df_v2.at[_index_eps,'ElectiveAdmission']=1

In [86]:
def addPositiveRastreios(data_frame, col):
    df_v2[col] = 0
    eps_list = list(data_frame['EpisodioKey'])
    for _index_eps, ep in enumerate(df['Key']):
        if ep in eps_list:
            df_v2.at[_index_eps,col]=1

In [87]:
def addBloodGases(data_frame, col, _type):
    df_v2[col] = np.nan
    eps_list = list(data_frame['Key'])
    for _index_eps, ep in enumerate(df['Key']):
        if ep in eps_list:
            _index = eps_list.index(ep)
            df_v2.at[_index_eps,col]=data_frame[_type][_index]

In [88]:
#df2 = OnlyPneuAdmission(df, df_infections_pneu)

#df2 = df2[ df2['DoenteIdadeEmAnos'] > 18]

#df2 = df2[ df2['Key'] <  15348]  # Até Fev, 2020 (remover possibilidades de doentes covid)
#df2 = df2[ df2['SAPSCompleto'] == -1]
#df2 = df2.reset_index()
#df2.shape

In [89]:
"""
    Get features from Episodios table

# Convert the dictionary into DataFrame
#df_v2['Log_LOS'] = np.log(df_v2['TempoInternamentoEmDias'])

df_v2 = pd.DataFrame({'LOS': df['TempoInternamentoEmDias']})
#df_v2['Falecido']=df['Falecido']
df_v2['DoenteIdadeEmAnos']=df['DoenteIdadeEmAnos']
df_v2['TipoProvenienciaId']=df['TipoProvenienciaId']
df_v2['HospitalRegiaoId']=df['HospitalRegiaoId']

df_v2['HospitalCodigo']=df['HospitalCodigo']
# Group by 'column1' and 'column2' and assign the ID of each group to a new column
df_v2['PacienteUnicoId'] = df.groupby(['DoenteId', 'HospitalCodigo']).ngroup() + 1

df_v2['ServicoId']=df['ServicoId']
df_v2['DoenteSexoId']=df['DoenteSexoId']
df_v2['SAPSScore']=df['SAPSScore']
#df_v2['TipoServicoId']=df['TipoServicoId']
df_v2['TipoAdmissaoId']=df['TipoAdmissaoId']
df_v2['Traqueostomia']=df['Traqueostomia']
df_v2['TempoAteAdmissaoUni'] = (df['DataAdmissaoUnidade'] - df['DataAdmissaoHospitalar'])  / pd.to_timedelta(1, unit='D')

df_v2['Unique_patient_per_Hosp'] = df_v2.groupby(['HospitalCodigo'])['PacienteUnicoId'].transform('nunique')
df_v2['Unique_patient_per_Admission'] = df_v2.groupby(['TipoAdmissaoId'])['PacienteUnicoId'].transform('nunique')
df_v2['Unique_patient_per_Service'] = df_v2.groupby(['ServicoId'])['PacienteUnicoId'].transform('nunique')
df_v2 = df_v2.drop('PacienteUnicoId', axis=1)
df_v2.shape"""

"\n    Get features from Episodios table\n\n# Convert the dictionary into DataFrame\n#df_v2['Log_LOS'] = np.log(df_v2['TempoInternamentoEmDias'])\n\ndf_v2 = pd.DataFrame({'LOS': df['TempoInternamentoEmDias']})\n#df_v2['Falecido']=df['Falecido']\ndf_v2['DoenteIdadeEmAnos']=df['DoenteIdadeEmAnos']\ndf_v2['TipoProvenienciaId']=df['TipoProvenienciaId']\ndf_v2['HospitalRegiaoId']=df['HospitalRegiaoId']\n\ndf_v2['HospitalCodigo']=df['HospitalCodigo']\n# Group by 'column1' and 'column2' and assign the ID of each group to a new column\ndf_v2['PacienteUnicoId'] = df.groupby(['DoenteId', 'HospitalCodigo']).ngroup() + 1\n\ndf_v2['ServicoId']=df['ServicoId']\ndf_v2['DoenteSexoId']=df['DoenteSexoId']\ndf_v2['SAPSScore']=df['SAPSScore']\n#df_v2['TipoServicoId']=df['TipoServicoId']\ndf_v2['TipoAdmissaoId']=df['TipoAdmissaoId']\ndf_v2['Traqueostomia']=df['Traqueostomia']\ndf_v2['TempoAteAdmissaoUni'] = (df['DataAdmissaoUnidade'] - df['DataAdmissaoHospitalar'])  / pd.to_timedelta(1, unit='D')\n\ndf_v2[

In [90]:
%%time
"""
    Add most significant variables

addTipoPrescricoes(df_prescricoes_24h)
addNumPresc(df_prescricoes_24h)
addDuracaoPresc(df_prescricoes_24h)
addVMI(df_ventilacao_24h)
addNumeroDiagn()
addNumeroDiagnAntec()
addDispositivosNum(df_dispositivosI_24h, df_dispositivosII_24h)
#addDevice(df_dispositivosI_24h, df_dispositivosII_24h, 'Tubo Orotraqueal')
addGlasgowScore(df_glasgow_24h_min)
#addPosicaoProne(df_pos_prone_24h)

# Pr Celeste

addDiseaseAntec(df_diabetes, 'DiagnsAntecendentes')
addDiseaseAntec(df_doenca_resp, 'DiagnsAntecendentes')
addDiseaseAntec(df_doenca_circ, 'DiagnsAntecendentes')
#addAlcoTaba(df_alcoolismo_ant, 'DiagnsAntecendentes')
#addAlcoTaba(df_tabagismo_ant, 'DiagnsAntecendentes')

# --------------- SAPS -----------------
#saps_metrics = ['FC', 'Temp', 'TMed', 'TDias', 'TSist', 'HCo3', 'FiO2', 'Na', 'K', 'Glasgow', 'Idade', 'TipoAdmissao', 'ProbMorte'
#                , 'Ureia', 'DoencasCronicas', 'Bilirrubina', 'DebitoUrinario', 'Leucocitos', 'PaO2', 'DoenteVent', 'PaoFio']

#saps_metrics = ['FiO2', 'Na'
#               , 'Ureia', 'DoencasCronicas', 'Bilirrubina', 'DebitoUrinario', 'Leucocitos', 'DoenteVent', 'PaoFio']

saps_metrics = ['DoencasCronicas', 'PaoFio']

for metric in saps_metrics:
    addSAPSmetrics(metric)

# --------------- Sinais Vitais -----------------
addVitalSign(df_sinais_monitor_FC_24h_max, df_sinais_enf_FC_24h, 'FC')      # dataframe monitor , dataframe enfermagen, nome da feature
#addVitalSign(df_sinais_monitor_FiO2_24h_min, df_sinais_enf_FiO2_24h, 'FiO2')      # dataframe monitor , dataframe enfermagen, nome da feature
#addVitalSign(df_sinais_monitor_Temp_24h_max, df_sinais_enf_Temp_24h, 'Temp')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_FR_24h_max, df_sinais_enf_FR_24h, 'FR')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_SpO2_24h_min, df_sinais_enf_SpO2_24h, 'SpO2')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_TAM_24h_min, df_sinais_enf_TAM_24h, 'TAM')      # dataframe monitor , dataframe enfermagen, nome da feature

# --------------- Gasimetrias -----------------
_type = 'MinValor'
addBloodGases(df_gas_be, 'BE_min', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_min', _type)
addBloodGases(df_gas_fio2, 'FiO2_min', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_min', _type)
addBloodGases(df_gas_methb, 'metHb_min', _type)
addBloodGases(df_gas_paco2, 'PaCO2_min', _type)
addBloodGases(df_gas_pao2, 'PaO2_min', _type)
addBloodGases(df_gas_sao2, 'SaO2_min', _type)
addBloodGases(df_gas_ph, 'pH_min', _type)

_type = 'MaxValor'
addBloodGases(df_gas_be, 'BE_max', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_max', _type)
addBloodGases(df_gas_fio2, 'FiO2_max', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_max', _type)
addBloodGases(df_gas_methb, 'metHb_max', _type)
addBloodGases(df_gas_paco2, 'PaCO2_max', _type)
addBloodGases(df_gas_pao2, 'PaO2_max', _type)
addBloodGases(df_gas_sao2, 'SaO2_max', _type)
addBloodGases(df_gas_ph, 'pH_max', _type)

_type = 'AvgValor'
addBloodGases(df_gas_be, 'BE_avg', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_avg', _type)
#addBloodGases(df_gas_fio2, 'FiO2_avg', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_avg', _type)
addBloodGases(df_gas_methb, 'metHb_avg', _type)
addBloodGases(df_gas_paco2, 'PaCO2_avg', _type)
addBloodGases(df_gas_pao2, 'PaO2_avg', _type)
addBloodGases(df_gas_sao2, 'SaO2_avg', _type)
addBloodGases(df_gas_ph, 'pH_avg', _type)"""

# --------------- Rastreios Positivos -----------------
#addPositiveRastreios(df_rastreios_micro_positivo, 'RastreiosMicroPositive')
#addPositiveRastreios(df_rastreios_sero_positivo, 'RastreiosSeroPositive')

addSepsis(df_infections_24h)
#addBacteriemia(df_infections_24h)

#addDiseaseAntec(df_diabetes, 'Diabetes')

#addAllDevices(df_dispositivosI_24h, df_dispositivosII_24h)

SyntaxError: invalid syntax (<unknown>, line 44)

In [ ]:
df_v2.head().T

In [ ]:
%%time
"""
    Add variables
"""
addTipoPrescricoes(df_prescricoes_24h)
addNumPresc(df_prescricoes_24h)
addDuracaoPresc(df_prescricoes_24h)
addVMI(df_ventilacao_24h)
addNumeroDiagn()

addDispositivosNum(df_dispositivosI_24h, df_dispositivosII_24h)
#addDevice(df_dispositivosI_24h, df_dispositivosII_24h, 'Tubo Orotraqueal')

addGlasgowScore(df_glasgow_24h_min)
addPosicaoProne(df_pos_prone_24h)

# Pr Celeste

addDiseaseAntec(df_diabetes, 'DiagnsAntecendentes')
addDiseaseAntec(df_doenca_resp, 'DiagnsAntecendentes')
addDiseaseAntec(df_doenca_circ, 'DiagnsAntecendentes')
addAlcoTaba(df_alcoolismo_ant, 'DiagnsAntecendentes')
addAlcoTaba(df_tabagismo_ant, 'DiagnsAntecendentes')

# --------------- SAPS -----------------
saps_metrics = ['HCo3', 'FiO2', 'Na', 'K', 'Ureia', 'DoencasCronicas', 'Bilirrubina', 'DebitoUrinario', 'Leucocitos', 'PaO2', 'PaoFio']

#saps_metrics = ['FiO2', 'Na'
#                , 'Ureia', 'DoencasCronicas', 'Bilirrubina', 'DebitoUrinario', 'Leucocitos', 'DoenteVent', 'PaoFio']

for metric in saps_metrics:
    addSAPSmetrics(metric)

# --------------- Sinais Vitais -----------------
addVitalSign(df_sinais_monitor_FC_24h_max, df_sinais_enf_FC_24h_max, 'FC_max')      # dataframe monitor , dataframe enfermagen, nome da feature
#addVitalSign(df_sinais_monitor_FiO2_24h_min, df_sinais_enf_FiO2_24h_min, 'FiO2_min')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_Temp_24h_max, df_sinais_enf_Temp_24h_max, 'Temp_max')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_FR_24h_max, df_sinais_enf_FR_24h_max, 'FR_max')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_SpO2_24h_min, df_sinais_enf_SpO2_24h_min, 'SpO2_min')      # dataframe monitor , dataframe enfermagen, nome da feature
addVitalSign(df_sinais_monitor_TAM_24h_min, df_sinais_enf_TAM_24h_min, 'TAM_min')      # dataframe monitor , dataframe enfermagen, nome da feature

# --------------- Gasimetrias -----------------
_type = 'MinValor'
addBloodGases(df_gas_be, 'BE_min', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_min', _type)
#addBloodGases(df_gas_fio2, 'FiO2_min', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_min', _type)
addBloodGases(df_gas_methb, 'metHb_min', _type)
addBloodGases(df_gas_paco2, 'PaCO2_min', _type)
addBloodGases(df_gas_pao2, 'PaO2_min', _type)
addBloodGases(df_gas_sao2, 'SaO2_min', _type)
addBloodGases(df_gas_ph, 'pH_min', _type)

_type = 'MaxValor'
addBloodGases(df_gas_be, 'BE_max', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_max', _type)
#addBloodGases(df_gas_fio2, 'FiO2_max', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_max', _type)
addBloodGases(df_gas_methb, 'metHb_max', _type)
addBloodGases(df_gas_paco2, 'PaCO2_max', _type)
addBloodGases(df_gas_pao2, 'PaO2_max', _type)
addBloodGases(df_gas_sao2, 'SaO2_max', _type)
addBloodGases(df_gas_ph, 'pH_max', _type)

_type = 'AvgValor'
addBloodGases(df_gas_be, 'BE_avg', _type)
addBloodGases(df_gas_bicarbonato, 'Bicarbonato_avg', _type)
#addBloodGases(df_gas_fio2, 'FiO2_avg', _type)
addBloodGases(df_gas_lactacto, 'Lactacto_avg', _type)
addBloodGases(df_gas_methb, 'metHb_avg', _type)
addBloodGases(df_gas_paco2, 'PaCO2_avg', _type)
addBloodGases(df_gas_pao2, 'PaO2_avg', _type)
addBloodGases(df_gas_sao2, 'SaO2_avg', _type)
addBloodGases(df_gas_ph, 'pH_avg', _type)

# --------------- Rastreios Positivos -----------------
addPositiveRastreios(df_rastreios_micro_positivo, 'RastreiosMicroPositive')
addPositiveRastreios(df_rastreios_sero_positivo, 'RastreiosSeroPositive')

addSepsis(df_infections_24h)
addBacteriemia(df_infections_24h)

addAllDevices(df_dispositivosI_24h, df_dispositivosII_24h)
df_v2.shape[0], df_v2.shape[1]

### Save Dataset

In [ ]:
df_v2.shape[0], df_v2.shape[1]
# Save the DataFrame to an Excel file
_name_file = 'DataFrames/' + str(df_v2.shape[0]) + '_' + str(df_v2.shape[1]) + '__allfinal_variables.xlsx'
df_v2.to_excel(_name_file, index=False)

In [ ]:
df_v2.shape[0], df_v2.shape[1]